### Introduction to Machine Learning, UZH FS18, Group Project

### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch

#     
# I.     Basic Preparation of Datasets

In [1]:
# hide unnecessary warnings ("depreciation" of packages etc.)
import warnings
warnings.filterwarnings('ignore')

# import packages
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer

# Import datasets

### Market Data from Wharton

In [2]:
# Import "big" wharton file -> wharton_ml
wharton = pd.read_csv('Data/wharton_ml3.csv', sep = ',')

# Rename  columns in wharton file
wharton.rename(columns = {'date': 'DATE_w', 'PERMNO': 'PERMNO_w'}, inplace = True)

# SPREAD seems to contain no data (nan), so I'm calculating spread via BID - ASK 
wharton['SPREAD'] = wharton['BID'] - wharton['ASK']

display(wharton.head(5))

,PERMNO_w,DATE_w,NAICS,DCLRDT,DLAMT,DLPDT,PAYDT,RCRDDT,SHRFLG,DISTCD,...,CFACSHR,ALTPRC,SPREAD,ALTPRCDT,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10107,31/01/2006,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.0,28.15,0.02,31/01/2006,0.076482,0.040044,0.038938,0.076354,0.075150,0.025467
1,10107,28/02/2006,511210.0,14/12/2005,NaN,NaN,09/03/2006,17/02/2006,0.0,1232.0,...,1.0,26.87,-0.01,28/02/2006,-0.045471,-0.001637,-0.003552,0.004836,0.003476,0.000453
2,10107,31/03/2006,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.0,27.21,0.00,31/03/2006,0.012653,0.019053,0.017585,0.036978,0.035232,0.011065
3,10107,28/04/2006,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.0,24.15,0.00,28/04/2006,-0.112459,0.012965,0.011494,0.009791,0.008497,0.012187
4,10107,31/05/2006,511210.0,27/03/2006,NaN,NaN,08/06/2006,17/05/2006,0.0,1232.0,...,1.0,22.65,0.00,31/05/2006,-0.062112,-0.031045,-0.033025,-0.044331,-0.045982,-0.030917


### Financial Ratios (from OLAT)

In [3]:
# Import financial ratios (downloaded from OLAT)
ratios_all = pd.read_csv('Data/Ratios.csv', sep = ',', converters = {'permno': str})

# Rename columns in ratios file
ratios_all.rename(columns = {'permno': 'PERMNO_r', 'public_date': 'DATE_r'}, inplace = True)

# REMOVE DATA THAT IS (PBLY) NOT NEEDED
ratios_all = ratios_all.drop(['adate', 'qdate'], axis = 1)

display(ratios_all.head())

,PERMNO_r,DATE_r,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,10107,31/01/2006,33.861,0.176,15.245,23.074,23.264,23.856,23.856,7.211,...,1.296,0.151,0.025,0.0,0.055,6.019,11.538,1.14%,15.506,2.109
1,10107,28/02/2006,31.756,0.163,14.951,21.496,21.846,22.392,22.392,6.713,...,1.323,0.151,0.025,0.0,0.036,6.281,10.280,1.34%,14.555,1.838
2,10107,31/03/2006,31.820,0.163,14.951,21.768,22.122,22.675,22.675,6.727,...,1.323,0.151,0.025,0.0,0.036,6.293,10.410,1.32%,14.739,1.842
3,10107,30/04/2006,28.176,0.163,14.951,19.320,19.634,20.125,20.125,5.957,...,1.323,0.151,0.025,0.0,0.036,5.573,9.239,1.49%,13.081,1.666
4,10107,31/05/2006,25.921,0.151,15.120,17.695,17.835,17.976,17.976,5.419,...,1.388,0.150,0.025,0.0,0.024,5.496,0.709,1.59%,-5.842,1.480


# Data preparation

### Convert Date and the PERMNO - Code, then rename columns

In [4]:
### DATE
# set date as datetime64
wharton['DATE_w'] = pd.to_datetime(wharton['DATE_w']).dt.date.astype('datetime64[ns]')
ratios_all['DATE_r'] = pd.to_datetime(ratios_all['DATE_r']).dt.date.astype('datetime64[ns]')

### PERMNO
# set PERMNO column as integer (int64) for ratios_all, because its type 'O'
ratios_all['PERMNO_r'] = ratios_all['PERMNO_r'].convert_objects(convert_numeric = True)

### RENAME COLUMNS AND SET PERMNO AND DATE AS INDEX
wharton.rename(columns = {'PERMNO_w': 'PERMNO', 'DATE_w': 'DATE'}, inplace = True)
ratios_all.rename(columns = {'PERMNO_r': 'PERMNO', 'DATE_r': 'DATE'}, inplace = True)


### Calculate Returns for each Company (PERMNO-group)

In [5]:
# make empty dataframe
grouped_by_permno = pd.DataFrame()

# Now follows a typical split-change-merge pattern to calculate the values for each PERMNO
# For each permno in df, do:
for df_key in wharton.groupby('PERMNO').groups:
    permno_group = wharton.groupby('PERMNO').get_group(df_key)
    permno_group['PRC_RET'] = np.log(permno_group['PRC']/ permno_group['PRC'].shift(1))
    grouped_by_permno = pd.concat([grouped_by_permno, permno_group])
wharton = grouped_by_permno

# Delete unused variables
del df_key, permno_group, grouped_by_permno

display(wharton['PRC_RET'].head())

0         NaN
1   -0.046537
2    0.012574
3   -0.119300
4   -0.064125
Name: PRC_RET, dtype: float64

### Convert calculated returns to movement

In [6]:
### Set 0, 1 for PRC_RET
def set_mov(mydata):
    if mydata['PRC_RET'] > 0:
        return 'UP'
    elif mydata['PRC_RET'] < 0:
        return 'DOWN'
    elif mydata['PRC_RET'] == 0:
        return 'UP'

wharton = wharton.assign(MOVEMENT = wharton.apply(set_mov, axis = 1))
wharton['PRC_MOV'] = wharton['MOVEMENT'].factorize()[0]
wharton = wharton.drop('MOVEMENT', axis =1)

display(wharton.PRC_MOV.head())

0   -1
1    0
2    1
3    0
4    0
Name: PRC_MOV, dtype: int64

### Set all dates to first day of month

In [7]:
# Because wharton and ratios dataset have sometimes differing "end" dates of month
wharton['DATE'] = wharton['DATE'].apply(lambda dt: dt.replace(day=1))
ratios_all['DATE'] = ratios_all['DATE'].apply(lambda dt: dt.replace(day=1))


## Join the two prepared datasets (outer join)

In [8]:
# Set index
wharton = wharton.set_index(['PERMNO', 'DATE'])
ratios_all = ratios_all.set_index(['PERMNO', 'DATE'])

# join datasets
joined_dataset = wharton.join(ratios_all, how = 'outer')

# reset index
joined_dataset = joined_dataset.reset_index()

display(joined_dataset.head())



,PERMNO,DATE,NAICS,DCLRDT,DLAMT,DLPDT,PAYDT,RCRDDT,SHRFLG,DISTCD,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,10107,2006-01-01,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.296,0.151,0.025,0.0,0.055,6.019,11.538,1.14%,15.506,2.109
1,10107,2006-02-01,511210.0,14/12/2005,NaN,NaN,09/03/2006,17/02/2006,0.0,1232.0,...,1.323,0.151,0.025,0.0,0.036,6.281,10.280,1.34%,14.555,1.838
2,10107,2006-03-01,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.323,0.151,0.025,0.0,0.036,6.293,10.410,1.32%,14.739,1.842
3,10107,2006-04-01,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.323,0.151,0.025,0.0,0.036,5.573,9.239,1.49%,13.081,1.666
4,10107,2006-05-01,511210.0,27/03/2006,NaN,NaN,08/06/2006,17/05/2006,0.0,1232.0,...,1.388,0.150,0.025,0.0,0.024,5.496,0.709,1.59%,-5.842,1.480


### Extract all rows of each PERMNO-Group except for first one (because it is Nan)

In [9]:
# Prepare empty dataframe
grouped_by_permno = pd.DataFrame()

# Group by PERMNO-Code and then remove first (0) row because it is nan in every PRC_RET group
for df_key in joined_dataset.groupby('PERMNO').groups:
    permno_group = joined_dataset.groupby('PERMNO').get_group(df_key)
    permno_group = permno_group[1:]
    grouped_by_permno = pd.concat([grouped_by_permno, permno_group])
joined_dataset = grouped_by_permno

# Delete unused variables
del df_key, permno_group, grouped_by_permno

display(joined_dataset['PRC_RET'].head())

1   -0.046537
2    0.012574
3   -0.119300
4   -0.064125
5    0.028294
Name: PRC_RET, dtype: float64

### Prepare column "divyield"

In [10]:
display(joined_dataset['divyield'].head())

# Remove percentages in row "divyield" and divide with 100 (so its decimal percentage) with string split
joined_dataset['divyield'] = joined_dataset['divyield'].str.rstrip('%').astype('float')/100

display(joined_dataset['divyield'].head())

1    1.34%
2    1.32%
3    1.49%
4    1.59%
5    1.55%
Name: divyield, dtype: object

1    0.0134
2    0.0132
3    0.0149
4    0.0159
5    0.0155
Name: divyield, dtype: float64

### Prepare Column with lagged Returns for Response Vector

In [11]:
# New Column with lagged movement of prices, because we want to have lagged prices as response vector later
# because we want to forecast prices of tomorrow, not of today with our model
joined_dataset['NEXT_DAY_PREDICTION'] = joined_dataset['PRC_MOV']
grouped_by_permno = pd.DataFrame()

for df_key in joined_dataset.groupby('PERMNO').groups:
    permno_group = joined_dataset.groupby('PERMNO').get_group(df_key)
    permno_group['NEXT_DAY_PREDICTION'] = permno_group['NEXT_DAY_PREDICTION'].shift(-1)
    permno_group = permno_group[pd.notnull(permno_group['NEXT_DAY_PREDICTION'])]
    grouped_by_permno = pd.concat([grouped_by_permno, permno_group])
joined_dataset = grouped_by_permno

# Delete unused variables
del df_key, permno_group, grouped_by_permno

display(joined_dataset['NEXT_DAY_PREDICTION'].head())

1    1.0
2    0.0
3    0.0
4    1.0
5    1.0
Name: NEXT_DAY_PREDICTION, dtype: float64

### Change Datatype of "Date" (NOT SURE, COULD ALSO MAKE DIFFERENT!!!, LIKE IF WE TAKE SEASONALITY)

In [12]:
# Make date numeric because RF can not use otherwise
joined_dataset['DATE'] = joined_dataset['DATE'].astype('int64')


## Drop Columns & create different Versions and save the datafiles

#### Explanation of column names:
#### sprtrn;      S&P 500 Composite Index Return
#### SHROUT;    Number of Shares Outstanding
#### 

In [13]:
# REMOVE DATA THAT IS (PBLY) NOT NEEDED
joined_dataset = joined_dataset.drop(['DATE', 'PRC', 'PRC_RET', 'PRC_MOV','RET', 'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'DLAMT', 'DLPDT', 'DCLRDT', 
                                      'DLRETX', 'DLRET', 'DLPRC', 'PAYDT', 'RCRDDT', 'SHRFLG', 'DISTCD', 'DIVAMT', 'FACPR', 
                            'ACPERM', 'ACCOMP', 'SHRENDDT', 'FACSHR', 'ALTPRCDT', 'ALTPRC', 'CFACPR', 'CFACSHR'], axis = 1)

display(joined_dataset.head())
display(list(joined_dataset.columns.values))

,PERMNO,NAICS,BIDLO,ASKHI,VOL,BID,ASK,SHROUT,SPREAD,sprtrn,...,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward,NEXT_DAY_PREDICTION
1,10107,511210.0,26.39,28.0400,11088149.0,26.87,26.88,10333369.0,-0.01,0.000453,...,0.151,0.025,0.0,0.036,6.281,10.280,0.0134,14.555,1.838,1.0
2,10107,511210.0,26.85,27.8900,14514337.0,27.24,27.24,10225000.0,0.00,0.011065,...,0.151,0.025,0.0,0.036,6.293,10.410,0.0132,14.739,1.842,0.0
3,10107,511210.0,24.15,27.7400,14689919.0,24.16,24.16,10201203.0,0.00,0.012187,...,0.151,0.025,0.0,0.036,5.573,9.239,0.0149,13.081,1.666,0.0
4,10107,511210.0,22.56,24.2900,23651189.0,22.70,22.70,10201203.0,0.00,-0.030917,...,0.150,0.025,0.0,0.024,5.496,0.709,0.0159,-5.842,1.480,1.0
5,10107,511210.0,21.51,23.4702,19980809.0,23.38,23.31,10062000.0,0.07,0.000087,...,0.150,0.025,0.0,0.024,5.577,0.730,0.0155,-6.010,1.522,1.0


['PERMNO',
 'NAICS',
 'BIDLO',
 'ASKHI',
 'VOL',
 'BID',
 'ASK',
 'SHROUT',
 'SPREAD',
 'sprtrn',
 'CAPEI',
 'bm',
 'evm',
 'pe_op_basic',
 'pe_op_dil',
 'pe_exi',
 'pe_inc',
 'ps',
 'pcf',
 'dpr',
 'npm',
 'opmbd',
 'opmad',
 'gpm',
 'ptpm',
 'cfm',
 'roa',
 'roe',
 'roce',
 'efftax',
 'aftret_eq',
 'aftret_invcapx',
 'aftret_equity',
 'pretret_noa',
 'pretret_earnat',
 'GProf',
 'equity_invcap',
 'debt_invcap',
 'totdebt_invcap',
 'capital_ratio',
 'int_debt',
 'int_totdebt',
 'cash_lt',
 'invt_act',
 'rect_act',
 'debt_at',
 'debt_ebitda',
 'short_debt',
 'curr_debt',
 'lt_debt',
 'profit_lct',
 'ocf_lct',
 'cash_debt',
 'fcf_ocf',
 'lt_ppent',
 'dltt_be',
 'debt_assets',
 'debt_capital',
 'de_ratio',
 'intcov',
 'intcov_ratio',
 'cash_ratio',
 'quick_ratio',
 'curr_ratio',
 'cash_conversion',
 'inv_turn',
 'at_turn',
 'rect_turn',
 'pay_turn',
 'sale_invcap',
 'sale_equity',
 'sale_nwc',
 'rd_sale',
 'adv_sale',
 'staff_sale',
 'accrual',
 'ptb',
 'PEG_trailing',
 'divyield',
 'PEG

### Version 1: create dataset where missing values are filled with 'mean' => Imputed Dataset

In [14]:
### Fill missing value with sklearn IMPUTER (fills with mean) instead of removing all Nan
imp = Imputer(missing_values=np.nan, strategy = 'mean' , axis=0)
imputed_dataset = pd.DataFrame(imp.fit_transform(joined_dataset))
imputed_dataset.columns = joined_dataset.columns
imputed_dataset.index = joined_dataset.index

display(imputed_dataset.isnull().sum())

imputed_dataset.to_csv('Data/generated/imputed_dataset_ml.csv')

display(imputed_dataset.head())

PERMNO                 0
NAICS                  0
BIDLO                  0
ASKHI                  0
VOL                    0
BID                    0
ASK                    0
SHROUT                 0
SPREAD                 0
sprtrn                 0
CAPEI                  0
bm                     0
evm                    0
pe_op_basic            0
pe_op_dil              0
pe_exi                 0
pe_inc                 0
ps                     0
pcf                    0
dpr                    0
npm                    0
opmbd                  0
opmad                  0
gpm                    0
ptpm                   0
cfm                    0
roa                    0
roe                    0
roce                   0
efftax                 0
                      ..
cash_debt              0
fcf_ocf                0
lt_ppent               0
dltt_be                0
debt_assets            0
debt_capital           0
de_ratio               0
intcov                 0
intcov_ratio           0


,PERMNO,NAICS,BIDLO,ASKHI,VOL,BID,ASK,SHROUT,SPREAD,sprtrn,...,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward,NEXT_DAY_PREDICTION
1,10107.0,511210.0,26.39,28.0400,11088149.0,26.87,26.88,10333369.0,-0.01,0.000453,...,0.151,0.025,0.0,0.036,6.281,10.280,0.0134,14.555,1.838,1.0
2,10107.0,511210.0,26.85,27.8900,14514337.0,27.24,27.24,10225000.0,0.00,0.011065,...,0.151,0.025,0.0,0.036,6.293,10.410,0.0132,14.739,1.842,0.0
3,10107.0,511210.0,24.15,27.7400,14689919.0,24.16,24.16,10201203.0,0.00,0.012187,...,0.151,0.025,0.0,0.036,5.573,9.239,0.0149,13.081,1.666,0.0
4,10107.0,511210.0,22.56,24.2900,23651189.0,22.70,22.70,10201203.0,0.00,-0.030917,...,0.150,0.025,0.0,0.024,5.496,0.709,0.0159,-5.842,1.480,1.0
5,10107.0,511210.0,21.51,23.4702,19980809.0,23.38,23.31,10062000.0,0.07,0.000087,...,0.150,0.025,0.0,0.024,5.577,0.730,0.0155,-6.010,1.522,1.0


### Version 2: create dataset where rows with missing values are dropped  => Dropnan Dataset

In [15]:
dropnan_dataset = joined_dataset.dropna()

display(dropnan_dataset.isnull().sum())


# Save 
dropnan_dataset.to_csv('Data/generated/dropnan_dataset_ml.csv')

PERMNO                 0
NAICS                  0
BIDLO                  0
ASKHI                  0
VOL                    0
BID                    0
ASK                    0
SHROUT                 0
SPREAD                 0
sprtrn                 0
CAPEI                  0
bm                     0
evm                    0
pe_op_basic            0
pe_op_dil              0
pe_exi                 0
pe_inc                 0
ps                     0
pcf                    0
dpr                    0
npm                    0
opmbd                  0
opmad                  0
gpm                    0
ptpm                   0
cfm                    0
roa                    0
roe                    0
roce                   0
efftax                 0
                      ..
cash_debt              0
fcf_ocf                0
lt_ppent               0
dltt_be                0
debt_assets            0
debt_capital           0
de_ratio               0
intcov                 0
intcov_ratio           0


### Version 3: create dataset where nan are interpolated  => Interpolated Dataset

In [16]:

display(joined_dataset.isnull().sum())


PERMNO                    0
NAICS                     1
BIDLO                     0
ASKHI                     0
VOL                       0
BID                       0
ASK                       0
SHROUT                    0
SPREAD                    0
sprtrn                    0
CAPEI                    11
bm                       11
evm                       3
pe_op_basic               4
pe_op_dil                 4
pe_exi                    4
pe_inc                    4
ps                        3
pcf                       6
dpr                       9
npm                       3
opmbd                     3
opmad                     3
gpm                       3
ptpm                      3
cfm                       3
roa                       6
roe                      15
roce                     15
efftax                    9
                       ... 
cash_debt                15
fcf_ocf                 101
lt_ppent                121
dltt_be                  12
debt_assets         

In [17]:
interp_dataset = joined_dataset.groupby('PERMNO').apply(lambda group: group.interpolate(method='linear'))


In [18]:
display(interp_dataset.isnull().sum())

# Save
interp_dataset.to_csv('Data/generated/interp_dataset_ml.csv')

PERMNO                   0
NAICS                    1
BIDLO                    0
ASKHI                    0
VOL                      0
BID                      0
ASK                      0
SHROUT                   0
SPREAD                   0
sprtrn                   0
CAPEI                   11
bm                       2
evm                      2
pe_op_basic              2
pe_op_dil                2
pe_exi                   2
pe_inc                   2
ps                       2
pcf                      2
dpr                      8
npm                      2
opmbd                    2
opmad                    2
gpm                      2
ptpm                     2
cfm                      2
roa                      2
roe                      2
roce                    11
efftax                   8
                      ... 
cash_debt                2
fcf_ocf                 70
lt_ppent               120
dltt_be                  8
debt_assets              2
debt_capital             2
d

In [19]:
grouped_by_permno = pd.DataFrame()
df_key = 0
for df_key in joined_dataset.groupby('PERMNO').groups:
    permno_group = joined_dataset.groupby('PERMNO').get_group(df_key)
    permno_group = permno_group.interpolate(method = 'linear')
    grouped_by_permno = pd.concat([grouped_by_permno, permno_group])
jjj = grouped_by_permno

display(jjj.isnull().sum())



PERMNO                   0
NAICS                    1
BIDLO                    0
ASKHI                    0
VOL                      0
BID                      0
ASK                      0
SHROUT                   0
SPREAD                   0
sprtrn                   0
CAPEI                   11
bm                       2
evm                      2
pe_op_basic              2
pe_op_dil                2
pe_exi                   2
pe_inc                   2
ps                       2
pcf                      2
dpr                      8
npm                      2
opmbd                    2
opmad                    2
gpm                      2
ptpm                     2
cfm                      2
roa                      2
roe                      2
roce                    11
efftax                   8
                      ... 
cash_debt                2
fcf_ocf                 70
lt_ppent               120
dltt_be                  8
debt_assets              2
debt_capital             2
d

In [20]:
inn = joined_dataset.interpolate(method = 'linear')
display(inn.isnull().sum())

PERMNO                  0
NAICS                   0
BIDLO                   0
ASKHI                   0
VOL                     0
BID                     0
ASK                     0
SHROUT                  0
SPREAD                  0
sprtrn                  0
CAPEI                   0
bm                      0
evm                     0
pe_op_basic             0
pe_op_dil               0
pe_exi                  0
pe_inc                  0
ps                      0
pcf                     0
dpr                     0
npm                     0
opmbd                   0
opmad                   0
gpm                     0
ptpm                    0
cfm                     0
roa                     0
roe                     0
roce                    9
efftax                  0
                       ..
cash_debt               0
fcf_ocf                 0
lt_ppent                0
dltt_be                 6
debt_assets             0
debt_capital            0
de_ratio                0
intcov      